## These are the libraries I used:

In [0]:
#Install folium for interactive mapping and overlays
#!pip install folium

#Install geopandas to read the tectonic plate boundary geoJSON file
#!pip install geopandas

In [0]:
#Geopandas for reading geoJSON files
import geopandas as gpd

#Data analysis and visualization libraries
import pandas as pd
import numpy as np
%matplotlib inline
import time

#Datetime libraries
import datetime
from datetime import datetime
from dateutil.relativedelta import relativedelta

#Mapping libraries
import folium
from folium import plugins

#Library for popup window editing
import branca
from branca.element import *

#Import base64 for displaying large sets of data
import base64


In [0]:
#Specify base64 encoding to display the map instead of asking a web browser to download it

def _repr_html_(self, **kwargs):
    html = base64.b64encode(self.render(**kwargs).encode('utf8')).decode('utf8')
    onload = (
        'this.contentDocument.open();'
        'this.contentDocument.write(atob(this.getAttribute(\'data-html\')));'
        'this.contentDocument.close();'
    )
    if self.height is None:
        iframe = (
            '<div style="width:{width};">'
            '<div style="position:relative;width:100%;height:0;padding-bottom:{ratio};">'
            '<iframe src="about:blank" style="position:absolute;width:100%;height:100%;left:0;top:0;'
            'border:none !important;" '
            'data-html={html} onload="{onload}" '
            'allowfullscreen webkitallowfullscreen mozallowfullscreen>'
            '</iframe>'
            '</div></div>').format
        iframe = iframe(html=html, onload=onload, width=self.width, ratio=self.ratio)
    else:
        iframe = ('<iframe src="about:blank" width="{width}" height="{height}"'
                  'style="border:none !important;" '
                  'data-html={html} onload="{onload}" '
                  '"allowfullscreen" "webkitallowfullscreen" "mozallowfullscreen">'
                  '</iframe>').format
        iframe = iframe(html=html, onload=onload, width=self.width, height=self.height)
    return iframe

folium.branca.element.Figure._repr_html_ = _repr_html_

## Request seismic activity data around the Juan de Fuca plate from January 2010 to February 2020:

In [0]:
# Provide the data request URL and read it into a DataFrame
data_request_url1='https://earthquake.usgs.gov/fdsnws/event/1/query.csv?starttime=2010-01-01%2000%3A00%3A00&endtime=2020-02-23%2023%3A59%3A59&maxlatitude=51.895&minlatitude=38.486&maxlongitude=-123.267&minlongitude=-138.725&minmagnitude=2.5&eventtype=earthquake&orderby=time-asc&limit=20000'

rawdata=pd.read_csv(data_request_url1)

#Separate relevant variables from rawdata into a cleaner DataFrame
data =pd.DataFrame(data=None,columns=['Time','Latitude','Longitude','Magnitude','Location'])
data.Time=rawdata.time
data.Latitude=rawdata.latitude
data.Longitude=rawdata.longitude
data.Magnitude=rawdata.mag
data.Location=rawdata.place

What are the minimum, maximum, and average earthquake magnitudes in this data set?

In [30]:
averageMag = np.round(data.Magnitude.mean(),3)
maxMag = np.round(data.Magnitude.max(),3)
minMag = np.round(data.Magnitude.min(),3)
print('The magnitude of earthquakes in this data set ranges from '+str(minMag)+' to '+str(maxMag)+', with\nan average magnitude of '+str(averageMag)+'.')

The magnitude of earthquakes in this data set ranges from 2.5 to 6.8, with
an average magnitude of 3.287.


Retrieve plate boundary data and name a few major ones:

In [31]:
#Tectonic plate boundary locations taken from ArcGIS database: https://hub.arcgis.com/datasets/tga::plate-boundaries?geometry=172.178%2C9.944%2C-31.553%2C57.971
plateBoundaryURL='https://opendata.arcgis.com/datasets/1782b6616eb442e994839ff15e994e43_0.geojson'

#Read the GeoJSON file
plateBounds=gpd.read_file(plateBoundaryURL)

#Rename relevant fault lines
plateBounds['NAME'].iloc[165]='Mendocino Transform'
plateBounds['NAME'].iloc[317]='Gorda Ridge'
plateBounds['NAME'].iloc[303]='Juan de Fuca Ridge'
plateBounds['NAME'].iloc[326]='Cascadia Trench'
plateBounds['NAME'].iloc[310]='Blanco Transform'
plateBounds['NAME'].iloc[274]='Sovanco Transform'
plateBounds['NAME'].iloc[289]='Coaxial Segment'
plateBounds['NAME'].iloc[283]='Cobb Segment'
plateBounds['NAME'].iloc[303]='Cleft Segment'



/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


## Map tectonic activity around the Juan de Fuca Plate over the entire timeline:

In [0]:
points=[]
timez=[]
coordz=[]
for x in range(0,len(data)):
  Time=data['Time'].iloc[x]
  timestamp=datetime.strptime(data['Time'].iloc[x],'%Y-%m-%dT%H:%M:%S.%fZ')
  niceTime=datetime.strftime(timestamp,'%B %d, %Y %I:%M:%S%p')
  magnitude=data['Magnitude'].iloc[x]
  location=data['Location'].iloc[x]
  if (magnitude>0):
    if(magnitude<=2.5):
      color='clear'
      radius=0.0001
    elif(magnitude<=3.3):
      color='green'
      radius=1.5*magnitude
    elif(magnitude>3.3 and magnitude <=5):
      color='orange'
      radius=1.5*magnitude
    else:
      color='red'
      radius=1.5*magnitude
  coords=[data.iloc[x]['Longitude'],data.iloc[x]['Latitude']]
  table = """<div style="width:280px">
  <table style=\'width:100%\'>
    <tr>
      <th>Location</th>
      <td>{{kwargs['loc']}}</td>
    </tr>
    <tr>
      <th>Date</th>/
      <td>{{kwargs['timestamp']}}</td>
    </tr>
    <br>
    <tr>
      <th>Magnitude</th>
      <th>{{kwargs['mgtd']}}</th>
    </tr>
  </table>
  </div>
  """
  e=Element(table)
  table=e.render(loc=location,mgtd=magnitude,timestamp=niceTime)
  point={
      'time':Time,
      'coordinates':coords,
      'popup':table,
      'color':color,
      'timestamp':timestamp,
      'magnitude':magnitude,
      'radius':radius
      }
  points.append(point)
  timez.append(Time)
  coordz.append(coords)

In [33]:
#Reformat earthquake time series for mapping
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': point['coordinates'],
        },
        'properties': {
            'time': point['time'],
            'popup': point['popup'],
            'id': 'house',
            'icon': 'circle',
            'iconstyle': {
                'fillColor': point['color'],
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': point['radius'],
                'color':point['color'],
            },
            'color':point['color'],
            'radius':point['radius']
        }
    } for point in points
]

#Specify time range and coordinates of earthquake events for mapping
features.append(
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': coordz,
        },
        'properties': {
            'popup': point['popup'],
            'times':timez,
            'icon': 'circle',
            'iconstyle': {
                'fillColor': point['color'],
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': point['radius'],
                'color':point['color'],
            },
            'style': {'weight': 0},
            'id': 'man'
        }
    }
)

#Method for seafloor topography theme retrieved from Mat Pearce's code
ArcGIS_Ocean_Floor = "https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}"
ArcGIS_Attribution = "Tiles &copy; Source: ArcGIS"

#Create map object centered at the right location and properly bounded for easy viewing 
m = folium.Map(
    tiles=ArcGIS_Ocean_Floor,
    attr=ArcGIS_Attribution,
    location=[45, -127],
    zoom_start=5,
    min_lat=30,
    max_lat=56,
    min_lon=-140,
    max_lon=-114,
    prefer_canvas=True,
)

#Show time series of earthquake events with optional slider bar
feature2=folium.plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='P1D', 
    add_last_point=True,
    auto_play=True,
    loop=True,
    max_speed=100,
    loop_button=True,
    date_options='Y-MM-DD',
    time_slider_drag_update=True,
    duration='P1M',
)

#Create plate boundary plate map with name and boundary type
plates = folium.GeoJson(
    plateBounds,
    style_function=lambda x: {'color': 'darkblue'},
    highlight_function=lambda x: {'color':'yellow'},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['NAME','LABEL'],
        aliases=['Name:','Boundary Type:'],
        labels=True,
        sticky=False,
        ),
    name='Plate Boundaries',
    overlay=True,
    show=True,
    control=True,
)

#Allow for the option of seeing all seismic events in one overlay
fg = folium.FeatureGroup(name='All Earthquake Events',overlay=True,show=False,control=True)
for point in points:
  folium.CircleMarker(
  location=[point['coordinates'][1],point['coordinates'][0]],
  popup=point['popup'],
  radius = point['radius'],
  color = point['color'],
  fill=True,
  fill_color=point['color'],
  ).add_to(fg)

#Point out a few notable volcanoes in the area
volcanoes = folium.FeatureGroup(name='Volcanoes',overlay=True,show=True,control=True)
folium.Marker([46.52,-129.58],tooltip='CoAxial Segment Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes)
folium.Marker([45.95,-130],tooltip='Axial Seamount Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes)
folium.Marker([42.67,-126.78],tooltip='North Gorda Ridge Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes) 
folium.Marker([44.83,-130.30],tooltip='Cleft Segment Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes)

folium.vector_layers.Rectangle(
    [[int(m.min_lat)+5,int(m.min_lon)+1],
    [int(m.max_lat)-2,int(m.max_lon)-5]],
    color='maroon',
).add_to(m)

folium.plugins.MiniMap(toggle_display=True,tile_layer='Stamen WaterColor').add_to(m)

#Add features to map object
m.add_child(plates)
m.add_child(feature2)
m.add_child(volcanoes)
m.add_child(fg)

#Enable control of plate boundary and seismic event layer display
folium.LayerControl(collapsed=True,).add_to(m)
#m.save('fullSet.html') #Uncomment if you want to export the map as an html file
m._repr_html_()
m

Output hidden; open in https://colab.research.google.com to view.

## Map all activity around the Juan de Fuca plate in April 2015:

In [0]:
points2=[]
timez2=[]
coordz2=[]
for x in range(0,len(data)):
  Time=data['Time'].iloc[x]
  timestamp=datetime.strptime(data['Time'].iloc[x],'%Y-%m-%dT%H:%M:%S.%fZ')
  if (timestamp > datetime.strptime('2015-04-01-00:00:00','%Y-%m-%d-%H:%M:%S') and timestamp < datetime.strptime('2015-05-01-23:59:59','%Y-%m-%d-%H:%M:%S')):
    niceTime=datetime.strftime(timestamp,'%B %d, %Y %I:%M:%S%p')
    #Time=datetime.strptime(Time,'%Y-%m-%dT%H:%M:%S.%fZ')
    #Time=datetime.strftime(Time,'%Y-%m-%d')
    magnitude=data['Magnitude'].iloc[x]
    location=data['Location'].iloc[x]
    if (magnitude>0):
      if(magnitude<=2.5):
        color='clear'
        radius=0.0001
      elif(magnitude<=3.3):
        color='green'
        radius=1.5*magnitude
      elif(magnitude>3.3 and magnitude <=5):
        color='orange'
        radius=1.5*magnitude
      else:
        color='red'
        radius=1.5*magnitude
    coords=[data.iloc[x]['Longitude'],data.iloc[x]['Latitude']]
    table = """<div style="width:280px">
    <table style=\'width:100%\'>
      <tr>
        <th>Location</th>
        <td>{{kwargs['loc']}}</td>
      </tr>
      <tr>
        <th>Date</th>
        <td>{{kwargs['timestamp']}}</td>
      </tr>
      <br>
      <tr>
        <th>Magnitude</th>
        <th>{{kwargs['mgtd']}}</th>
      </tr>
    </table>
    </div>
    """
    e=Element(table)
    table=e.render(loc=location,mgtd=magnitude,timestamp=niceTime)
    point={
        'time':Time,
        'coordinates':coords,
        'popup':table,
        'color':color,
        'timestamp':timestamp,
        'magnitude':magnitude,
        'radius':radius
        }
      
    points2.append(point)
    timez2.append(Time)
    coordz2.append(coords)

In [35]:
#Reformat earthquake time series for mapping
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': point['coordinates'],
        },
        'properties': {
            'time': point['time'],
            'popup': point['popup'],
            'id': 'house',
            'icon': 'circle',
            'iconstyle': {
                'fillColor': point['color'],
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': point['radius'],
                'color':point['color'],
            },
            'color':point['color'],
            'radius':point['radius']
        }
    } for point in points2
]

#Specify time range and coordinates of earthquake events for mapping
timeline=timez2
timeline.append('2015-05-01T00:00:00.000Z')
timeline.append('2015-03-31T00:00:00.000Z')

coordList=coordz2
coordList.append([-130.961, 51.479])
coordList.append([-130.951, 51.479])

features.append(
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': coordList,
        },
        'properties': {
            'popup': point['popup'],
            'times':timeline,
            'icon': 'circle',
            'iconstyle': {
                'fillColor': point['color'],
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': point['radius'],
                'color':point['color'],
            },
            'style': {'weight': 0},
            'id': 'man'
        }
    }
)

#Method for seafloor topography theme retrieved from Mat Pearce's code
ArcGIS_Ocean_Floor = "https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}"
ArcGIS_Attribution = "Tiles &copy; Source: ArcGIS"

#Create map object centered at the right location and properly bounded for easy viewing 
m = folium.Map(
    tiles=ArcGIS_Ocean_Floor,
    attr=ArcGIS_Attribution,
    location=[45, -127],
    zoom_start=5,
    min_lat=30,
    max_lat=56,
    min_lon=-140,
    max_lon=-114,
    prefer_canvas=True,
)

#Show time series of earthquake events with optional slider bar
feature2=folium.plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='P1D', #or P1M or P1D or PT1H
    add_last_point=True,
    auto_play=True,
    loop=True,
    max_speed=10,
    loop_button=True,
    date_options='Y-MM-DD',
    time_slider_drag_update=True,
    duration='P1M',
)

#Create plate boundary plate map with name and boundary type
plates = folium.GeoJson(
    plateBounds,
    style_function=lambda x: {'color': 'darkblue'},
    highlight_function=lambda x: {'color':'yellow'},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['NAME','LABEL'],
        aliases=['Name:','Boundary Type:'],
        labels=True,
        sticky=False,
        ),
    name='Plate Boundaries',
    overlay=True,
    show=True,
    control=True,
)

#Allow for the option of seeing all seismic events in one overlay
fg = folium.FeatureGroup(name='All Earthquake Events',overlay=True,show=False,control=True)
for point in points:
  folium.CircleMarker(
  location=[point['coordinates'][1],point['coordinates'][0]],
  popup=point['popup'],
  radius = point['radius'],
  color = point['color'],
  fill=True,
  fill_color=point['color'],
  ).add_to(fg)

#Point out a few notable volcanoes in the area
volcanoes = folium.FeatureGroup(name='Volcanoes',overlay=True,show=True,control=True)
folium.Marker([46.52,-129.58],tooltip='CoAxial Segment Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes)
folium.Marker([45.95,-130],tooltip='Axial Seamount Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes)
folium.Marker([42.67,-126.78],tooltip='North Gorda Ridge Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes) 
folium.Marker([44.83,-130.30],tooltip='Cleft Segment Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes)

folium.vector_layers.Rectangle(
    [[int(m.min_lat)+5,int(m.min_lon)+1],
    [int(m.max_lat)-2,int(m.max_lon)-5]],
    color='maroon',
).add_to(m)

folium.plugins.MiniMap(toggle_display=True,tile_layer='Stamen WaterColor').add_to(m)

#Add features to map object
m.add_child(plates)
m.add_child(feature2)
m.add_child(volcanoes)
m.add_child(fg)

#Enable control of plate boundary and seismic event layer display
folium.LayerControl(collapsed=True,).add_to(m)

#m.save('april2015.html') #Uncomment if you want to export the map as an html file
m._repr_html_()
m

Output hidden; open in https://colab.research.google.com to view.

## Map all activity in the Gorda Ridge divergent boundary region:

In [36]:
#Reformat earthquake time series for mapping
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': point['coordinates'],
        },
        'properties': {
            'time': point['time'],
            'popup': point['popup'],
            'id': 'house',
            'icon': 'circle',
            'iconstyle': {
                'fillColor': point['color'],
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': point['radius'],
                'color':point['color'],
            },
            'color':point['color'],
            'radius':point['radius']
        }
    } for point in points
]

#Specify time range and coordinates of earthquake events for mapping
features.append(
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': coordz,
        },
        'properties': {
            'popup': point['popup'],
            'times':timez,
            'icon': 'circle',
            'iconstyle': {
                'fillColor': point['color'],
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': point['radius'],
                'color':point['color'],
            },
            'style': {'weight': 0},
            'id': 'man'
        }
    }
)

#Method for seafloor topography theme retrieved from Mat Pearce's code
ArcGIS_Ocean_Floor = "https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}"
ArcGIS_Attribution = "Tiles &copy; Source: ArcGIS"

#Create map object centered at the right location and properly bounded for easy viewing 
m = folium.Map(
    tiles=ArcGIS_Ocean_Floor,
    attr=ArcGIS_Attribution,
    location=[41.5, -127],
    zoom_start=8,
    min_lat=30,
    max_lat=56,
    min_lon=-140,
    max_lon=-114,
    prefer_canvas=True,
)

#Show time series of earthquake events with optional slider bar
feature2=folium.plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='P1D', #or P1M or P1D or PT1H
    add_last_point=True,
    auto_play=True,
    loop=True,
    max_speed=100,
    loop_button=True,
    date_options='Y-MM-DD',
    time_slider_drag_update=True,
    duration='P1M',
)

#Create plate boundary plate map with name and boundary type
plates = folium.GeoJson(
    plateBounds,
    style_function=lambda x: {'color': 'darkblue'},
    highlight_function=lambda x: {'color':'yellow'},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['NAME','LABEL'],
        aliases=['Name:','Boundary Type:'],
        labels=True,
        sticky=False,
        ),
    name='Plate Boundaries',
    overlay=True,
    show=True,
    control=True,
)

#Allow for the option of seeing all seismic events in one overlay
fg = folium.FeatureGroup(name='All Earthquake Events',overlay=True,show=False,control=True)
for point in points:
  folium.CircleMarker(
  location=[point['coordinates'][1],point['coordinates'][0]],
  popup=point['popup'],
  radius = point['radius'],
  color = point['color'],
  fill=True,
  fill_color=point['color'],
  ).add_to(fg)

#Point out a few notable volcanoes in the area
volcanoes = folium.FeatureGroup(name='Volcanoes',overlay=True,show=True,control=True)
folium.Marker([46.52,-129.58],tooltip='CoAxial Segment Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes)
folium.Marker([45.95,-130],tooltip='Axial Seamount Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes)
folium.Marker([42.67,-126.78],tooltip='North Gorda Ridge Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes) 
folium.Marker([44.83,-130.30],tooltip='Cleft Segment Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes)

folium.vector_layers.Rectangle(
    [[40.3,-128],
    [43.25,-126.5]],
    color='maroon',
).add_to(m)

folium.plugins.MiniMap(toggle_display=True,tile_layer='Stamen WaterColor',minimized=True).add_to(m)

#Add features to map object
m.add_child(plates)
m.add_child(feature2)
m.add_child(volcanoes)
m.add_child(fg)
m.add_child(folium.ClickForMarker(popup=None))

#Enable control of plate boundary and seismic event layer display
folium.LayerControl(collapsed=True,).add_to(m)

#m.save('divBound.html') #Uncomment if you want to export the map as an html file
m._repr_html_()
m

Output hidden; open in https://colab.research.google.com to view.

## Map all activity in the Mendocino Transform boundary region:

In [37]:
#Reformat earthquake time series for mapping
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': point['coordinates'],
        },
        'properties': {
            'time': point['time'],
            'popup': point['popup'],
            'id': 'house',
            'icon': 'circle',
            'iconstyle': {
                'fillColor': point['color'],
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': point['radius'],
                'color':point['color'],
            },
            'color':point['color'],
            'radius':point['radius']
        }
    } for point in points
]

#Specify time range and coordinates of earthquake events for mapping
features.append(
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': coordz,
        },
        'properties': {
            'popup': point['popup'],
            'times':timez,
            'icon': 'circle',
            'iconstyle': {
                'fillColor': point['color'],
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': point['radius'],
                'color':point['color'],
            },
            'style': {'weight': 0},
            'id': 'man'
        }
    }
)

#Method for seafloor topography theme retrieved from Mat Pearce's code
ArcGIS_Ocean_Floor = "https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}"
ArcGIS_Attribution = "Tiles &copy; Source: ArcGIS"

#Create map object centered at the right location and properly bounded for easy viewing 
m = folium.Map(
    tiles=ArcGIS_Ocean_Floor,
    attr=ArcGIS_Attribution,
    location=[40.5, -126],
    zoom_start=8,
    min_lat=30,
    max_lat=56,
    min_lon=-140,
    max_lon=-114,
    prefer_canvas=True,
)

#Show time series of earthquake events with optional slider bar
feature2=folium.plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='P1D',
    add_last_point=True,
    auto_play=True,
    loop=True,
    max_speed=100,
    loop_button=True,
    date_options='Y-MM-DD',
    time_slider_drag_update=True,
    duration='P1M',
)

#Create plate boundary plate map with name and boundary type
plates = folium.GeoJson(
    plateBounds,
    style_function=lambda x: {'color': 'darkblue'},
    highlight_function=lambda x: {'color':'yellow'},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['NAME','LABEL'],
        aliases=['Name:','Boundary Type:'],
        labels=True,
        sticky=False,
        ),
    name='Plate Boundaries',
    overlay=True,
    show=True,
    control=True,
)

#Allow for the option of seeing all seismic events in one overlay
fg = folium.FeatureGroup(name='All Earthquake Events',overlay=True,show=False,control=True)
for point in points:
  folium.CircleMarker(
  location=[point['coordinates'][1],point['coordinates'][0]],
  popup=point['popup'],
  radius = point['radius'],
  color = point['color'],
  fill=True,
  fill_color=point['color'],
  ).add_to(fg)

#Point out a few notable volcanoes in the area
volcanoes = folium.FeatureGroup(name='Volcanoes',overlay=True,show=True,control=True)
folium.Marker([46.52,-129.58],tooltip='CoAxial Segment Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes)
folium.Marker([45.95,-130],tooltip='Axial Seamount Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes)
folium.Marker([42.67,-126.78],tooltip='North Gorda Ridge Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes) 
folium.Marker([44.83,-130.30],tooltip='Cleft Segment Volcano',icon=folium.Icon(prefix='fa', icon='fa-fire',color='red')).add_to(volcanoes)

folium.vector_layers.Rectangle(
    [[40,-128],
    [41,-124]],
    color='maroon',
).add_to(m)

folium.plugins.MiniMap(toggle_display=True,tile_layer='Stamen WaterColor',minimized=True).add_to(m)

#Add features to map object
m.add_child(plates)
m.add_child(feature2)
m.add_child(volcanoes)
m.add_child(fg)
m.add_child(folium.ClickForMarker(popup=None))

#Enable control of plate boundary and seismic event layer display
folium.LayerControl(collapsed=True,).add_to(m)

#m.save('tranBound.html') #Uncomment if you want to export the map as an html file
m._repr_html_()
m

Output hidden; open in https://colab.research.google.com to view.